# Toronto Neighborhoods
## Import libraries

In [4]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import folium
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim

print("Libraries imported!")

Libraries imported!


## Setting up Data
The data is scraped from [this Wikipedia Page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M). Postal codes with no boroughs assigned are dropped and the column names and contents are modified into the desired format.

In [5]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(res)
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]

df.rename(columns={'Postal code':'PostalCode'},inplace=True)

df = df[df.Borough != 'Not assigned']
df = df.reset_index(drop=True)

df.Neighborhood = df.Neighborhood.str.replace(" /",",")

df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
df.shape

(103, 3)

## Importing Latitude and Longitudes
Using the geocoder package was unsuccessful, so we will just download the CSV file and merge it with our original dataframe.

In [7]:
lat_lng = pd.read_csv('http://cocl.us/Geospatial_data')
lat_lng.rename(columns={'Postal Code':'PostalCode'},inplace=True)
lat_lng.head()


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
df = df.merge(lat_lng)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [9]:
df.shape

(103, 5)

## Neighborhood Analysis
### Restrict Data
Let's restrict our data to only those that contain Toronto in the name of the Borough

In [19]:
df = df[df.Borough.str.contains("Toronto")]
df = df.reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [11]:
df.shape

(39, 5)

### Visualizing the Map
Let's create a folium map object with markers for each postal code that have popups with the names of the neighborhoods within them.

*Note: Folium Maps will not render properly in GitHub. Follow the link [here](https://nbviewer.jupyter.org/github/alexanderWhile/Coursera_Capstone/blob/master/toronto_neighborhoods.ipynb) to see the full output*

In [12]:
m = folium.Map(
    location=[43.6532, -79.3832],
    zoom_start=11)

for i in range(0,len(df)):
    folium.Marker([df.iloc[i]['Latitude'],df.iloc[i]['Longitude']],
        popup=df.iloc[i]['Neighborhood']
        ).add_to(m)

m

### Connect to Foursquare
Define credentials and version

In [13]:
CLIENT_ID = 'IA4SDU5HX0UHCL4VSZJDAHBXWJHJY4HPTFNBLWHG4YHYSLWH'
CLIENT_SECRET = '21PID34DCUTLYIWC2RRRRWMBKIE1ZUUXQKE2ZEAASQ4VIWX5'
VERSION = '20200402'

print("Client ID:",CLIENT_ID)
print("Client Secret:", CLIENT_SECRET)

print("Version:", VERSION)

Client ID: IA4SDU5HX0UHCL4VSZJDAHBXWJHJY4HPTFNBLWHG4YHYSLWH
Client Secret: 21PID34DCUTLYIWC2RRRRWMBKIE1ZUUXQKE2ZEAASQ4VIWX5
Version: 20200402


Let's look at the top venues near our first postal code's neighborhoods.

In [22]:
neighborhood = df.loc[0,'Neighborhood']
neighborhood_lat = df.loc[0,'Latitude']
neighborhood_lng = df.loc[0,'Longitude']

LIMIT = 100
radius = 500

print("Neighborhood(s):", neighborhood)
print("Latitude:", neighborhood_lat)
print("Longitude:", neighborhood_lng)

Neighborhood(s): Regent Park, Harbourfront
Latitude: 43.6542599
Longitude: -79.3606359


Define url and make request

In [41]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighborhood_lat,
    neighborhood_lng,
    radius,
    LIMIT)

results = requests.get(url).json()
print("Success!")

Success!


Define function for parsing venue category

In [36]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

print("Function Defined!")

Function Defined!


Gather venues and format into a clean data frame

In [39]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues = nearby_venues.loc[:,filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type,axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


Check the number of venues returned. We can see we have found 45 venues nearby.

In [40]:
nearby_venues.shape

(45, 4)

Let's make a map of this postal code and the nearby venues. 

In [53]:
neighborhood_map = folium.Map(
    location=[neighborhood_lat,neighborhood_lng],
    zoom_start=15)

folium.vector_layers.CircleMarker(
    [neighborhood_lat,neighborhood_lng],
    radius=10,
    color='red',
    popup=neighborhood,
    fill = True,
    fill_color='red',
    fill_opacity=0.6,
).add_to(neighborhood_map)

for lat, lng, label in zip(nearby_venues.lat, nearby_venues.lng, nearby_venues.categories):
    folium.vector_layers.CircleMarker(
        [lat,lng],
        radius=5,
        color='blue',
        popup=label,
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(neighborhood_map)

neighborhood_map